下载数据的命令
# download from eastmoney.com
python collector.py download_data --source_dir ~/.qlib/fund_data/source/cn_1d --region CN --start 2020-11-01 --end 2020-11-10 --delay 0.1 --interval 1d


# normalize
python ~/Github/qlib/scripts/data_collector/fund/collector.py normalize_data --source_dir ~/.qlib/fund_data/source/cn_1d --normalize_dir ~/.qlib/fund_data/source/cn_1d_nor --region CN --interval 1d --date_field_name FSRQ

# dump data
cd qlib/scripts

python ~/Github/qlib/scripts/dump_bin.py dump_all --csv_path ~/.qlib/fund_data/source/cn_1d_nor --qlib_dir ~/.qlib/qlib_data/cn_fund_data --freq day --date_field_name FSRQ --include_fields DWJZ,LJJZ

合并

## my code

python ~/Github/qlib/scripts/data_collector/fund/collector.py download_data --source_dir ~/.qlib/fund_data/source/cn_1d --region CN  --delay 0.1 --interval 1d

python ~/Github/qlib/scripts/data_collector/fund/collector.py normalize_data --source_dir ~/.qlib/fund_data/source/cn_1d --normalize_dir ~/.qlib/fund_data/source/cn_1d_nor --region CN --interval 1d --date_field_name FSRQ

python ~/Github/qlib/scripts/dump_bin.py dump_all --csv_path ~/.qlib/fund_data/source/cn_1d_nor --qlib_dir ~/.qlib/qlib_data/cn_fund_data --freq day --date_field_name FSRQ --include_fields DWJZ,LJJZ

In [1]:
import os
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 8.0) # 调整大小，可根据自实际情况进行设置
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['font.sans-serif']=['SimHei'] # 下面这两个是设置乱码的
plt.rcParams['axes.unicode_minus']=False
plt.style.use('ggplot') # 使用'ggplot'风格美化显示的图表


In [2]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from read_and_write_fund_code import *
import datetime

# Step1： 读出来所有的数据

In [3]:
import qlib
from qlib.data import D

qlib.init(provider_uri="~/.qlib/qlib_data/cn_fund_data")
df = D.features(D.instruments(market="all"), ["$DWJZ", "$LJJZ"], freq="day")
df = df.reset_index()
df_new = df.copy()

[56260:MainThread](2021-08-08 17:37:27,134) INFO - qlib.Initialization - [config.py:276] - default_conf: client.
[56260:MainThread](2021-08-08 17:37:27,136) WARNING - qlib.Initialization - [config.py:291] - redis connection failed(host=127.0.0.1 port=6379), cache will not be used!
[56260:MainThread](2021-08-08 17:37:27,296) INFO - qlib.Initialization - [__init__.py:46] - qlib successfully initialized based on client settings.
[56260:MainThread](2021-08-08 17:37:27,297) INFO - qlib.Initialization - [__init__.py:47] - data_path=/Users/wangfan/.qlib/qlib_data/cn_fund_data


# Step2 数据加工

计算未来一段时间的涨幅，作为预测的y

In [4]:
df_new["LJJZ_1d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-1)
df_new["LJJZ_5d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-5)
df_new["LJJZ_20d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-20)
df_new["LJJZ_60d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-60)
df_new["LJJZ_120d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-120)
df_new["LJJZ_240d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-240)
df_new["LJJZ_480d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-480)
#df_new["LJJZ_720d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-720)
df_new.index = range(len(df_new))
df_new["y_1d"]=100 * (df_new["LJJZ_1d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_5d"]=100 * (df_new["LJJZ_5d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_20d"]=100 * (df_new["LJJZ_20d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_60d"]=100 * (df_new["LJJZ_60d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_120d"]=100 * (df_new["LJJZ_120d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_240d"]=100 * (df_new["LJJZ_240d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_480d"]=100 * (df_new["LJJZ_480d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
#df_new["y_720d"]=100 * (df_new["LJJZ_720d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new = df_new.drop(columns = ['LJJZ_1d','LJJZ_5d','LJJZ_20d','LJJZ_60d','LJJZ_120d','LJJZ_240d','LJJZ_480d'])

构建特征：
- 过去的业绩表现

In [5]:
df_new["LJJZ_20d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(20)
df_new["LJJZ_60d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(60)
df_new["LJJZ_120d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(120)
df_new["LJJZ_240d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(240)
df_new["LJJZ_480d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(480)
#df_new["LJJZ_720d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(720)
df_new.index = range(len(df_new))

df_new["his_20d"]=-100 * (df_new["LJJZ_20d"]- df_new["$LJJZ"])/df_new["LJJZ_20d"]
df_new["his_60d"]=-100 * (df_new["LJJZ_60d"]- df_new["$LJJZ"])/df_new["LJJZ_60d"]
df_new["his_120d"]=-100 * (df_new["LJJZ_120d"]- df_new["$LJJZ"])/df_new["LJJZ_120d"]
df_new["his_240d"]=-100 * (df_new["LJJZ_240d"]- df_new["$LJJZ"])/df_new["LJJZ_240d"]
df_new["his_480d"]=-100 * (df_new["LJJZ_480d"]- df_new["$LJJZ"])/df_new["LJJZ_480d"]
#df_new["his_720d"]=-100 * (df_new["LJJZ_720d"]- df_new["$LJJZ"])/df_new["LJJZ_720d"]
df_new = df_new.drop(columns = ['LJJZ_20d','LJJZ_60d','LJJZ_120d','LJJZ_240d','LJJZ_480d'])

df_new["rank_20d"] = df_new.groupby(['datetime'])["his_20d"].rank(method='min',ascending=False)
df_new["rank_60d"] = df_new.groupby(['datetime'])["his_60d"].rank(method='min',ascending=False)
df_new["rank_120d"] = df_new.groupby(['datetime'])["his_120d"].rank(method='min',ascending=False)
df_new["rank_240d"] = df_new.groupby(['datetime'])["his_240d"].rank(method='min',ascending=False)
df_new["rank_480d"] = df_new.groupby(['datetime'])["his_480d"].rank(method='min',ascending=False)
#df_new["rank_720d"] = df_new.groupby(['datetime'])["his_720d"].rank(method='min',ascending=False)
df_new.index = range(len(df_new))

In [55]:
w1, w2, w3, w4, w5, w6 = 6, 5, 4, 3, 2, 1
df_new["average_rank"] = df_new["rank_20d"] * w1 + df_new["rank_60d"] * w2 + df_new["rank_120d"] * w3 + df_new["rank_240d"] * w4 + df_new["rank_480d"] * w5 # + df_new["rank_720d"] * w6
#df_train["average_rank"] = df_train["rank_120d"] + df_train["rank_240d"] + df_train["rank_480d"] + df_train["rank_720d"]
df_new["rank_of_average_rank"] = df_new.groupby(['datetime'])["average_rank"].rank(method='min')
df_new.index = range(len(df_new))
# rank越小，平均排名越高

###  missing value handling

### 训练集和验证集分开。

In [127]:
x_train = df_new[df_new['y_20d'].notnull()]
y_train= df_new['y_20d']
x_test = df_new[df_new['y_20d'].isnull()]
x_train = x_train.drop(columns = ['y_1d','y_5d','y_20d','y_60d','y_480d'])
x_test = x_test.drop(columns = ['y_1d','y_5d','y_20d','y_60d','y_480d'])
#df_test = df_test.dropna()

In [128]:
df_train.shape

(1811315, 26)

In [129]:
df_test.shape

(6297475, 20)

In [130]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [131]:
# Missing values statistics
missing_values = missing_values_table(df_test)
missing_values.head(20)

Your selected dataframe has 20 columns.
There are 18 columns that have missing values.


,Missing Values,% of Total Values
rank_of_average_rank,3720081,59.1
average_rank,3720081,59.1
rank_720d,3682593,58.5
his_720d,3682593,58.5
y_240d,3154662,50.1
his_480d,2927169,46.5
rank_480d,2927169,46.5
y_120d,2067426,32.8
his_240d,2055406,32.6
rank_240d,2055406,32.6


In [132]:
start_rank = 1 

end_rank = 10
start_date='2020-01-01'
end_date='2021-04-29'
df_new[(df_new['instrument'] == '001532')&(df_new['rank_of_average_rank']>=start_rank)
                &(df_new['rank_of_average_rank']<=end_rank)
                 &(df_new['datetime']>start_date)
                 &(df_new['datetime']<end_date)]

,instrument,datetime,$DWJZ,$LJJZ,y_1d,y_5d,y_20d,y_60d,y_120d,y_240d,...,his_480d,his_720d,rank_20d,rank_60d,rank_120d,rank_240d,rank_480d,rank_720d,average_rank,rank_of_average_rank
1543531,001532,2021-03-08,2.620,2.620,-1.946560,0.229009,3.854965,21.488558,NaN,NaN,...,128.222977,134.977570,347.0,127.0,164.0,69.0,139.0,85.0,3943.0,8.0
1543534,001532,2021-03-11,2.626,2.626,0.647371,1.713636,4.455450,20.639759,NaN,NaN,...,123.109589,145.650131,296.0,264.0,201.0,123.0,182.0,92.0,4725.0,7.0
1543535,001532,2021-03-12,2.643,2.643,-0.643207,0.227016,1.929630,21.415068,NaN,NaN,...,123.226341,146.318741,210.0,222.0,164.0,126.0,183.0,77.0,3847.0,10.0
1543536,001532,2021-03-15,2.626,2.626,-0.342720,1.713636,2.170608,20.868246,NaN,NaN,...,120.486984,141.360275,258.0,155.0,108.0,106.0,161.0,67.0,3462.0,9.0
1543538,001532,2021-03-17,2.641,2.641,1.135932,-2.234003,3.483529,16.584625,NaN,NaN,...,127.085129,143.410141,301.0,303.0,145.0,160.0,186.0,76.0,4829.0,10.0
1543539,001532,2021-03-18,2.671,2.671,-0.823664,-3.332085,2.845375,16.997379,NaN,NaN,...,126.932869,144.597061,283.0,213.0,159.0,163.0,196.0,78.0,4358.0,8.0
1543545,001532,2021-03-26,2.643,2.643,0.983732,2.421496,6.999629,24.404097,NaN,NaN,...,112.289146,139.836655,203.0,190.0,141.0,116.0,193.0,77.0,3543.0,8.0
1543554,001532,2021-04-09,2.743,2.743,-1.786366,0.145824,5.432006,27.342325,NaN,NaN,...,123.918365,153.044296,375.0,282.0,127.0,66.0,144.0,66.0,4720.0,7.0
1543557,001532,2021-04-14,2.728,2.728,0.183289,2.382700,8.027863,32.441349,NaN,NaN,...,125.454529,153.296188,732.0,226.0,121.0,61.0,131.0,64.0,6515.0,10.0
1543558,001532,2021-04-15,2.733,2.733,0.512255,2.817413,8.671789,29.381630,NaN,NaN,...,124.384232,151.889404,525.0,161.0,111.0,64.0,137.0,64.0,4929.0,7.0


# Step 3 模拟回测

In [133]:
df_train.columns

Index(['instrument', 'datetime', '$DWJZ', '$LJJZ', 'y_1d', 'y_5d', 'y_20d',
       'y_60d', 'y_120d', 'y_240d', 'y_480d', 'y_720d', 'his_20d', 'his_60d',
       'his_120d', 'his_240d', 'his_480d', 'his_720d', 'rank_20d', 'rank_60d',
       'rank_120d', 'rank_240d', 'rank_480d', 'rank_720d', 'average_rank',
       'rank_of_average_rank'],
      dtype='object')

In [134]:
def simulation_240d(start_rank,end_rank,start_date='1900-01-01',end_date='2021-04-29',dingtou=False,frequency=5):
    if not dingtou:
        return np.mean(df_train[(df_train['rank_of_average_rank']>=start_rank)
                &(df_train['rank_of_average_rank']<=end_rank)&(df_train['datetime']>start_date)&(df_train['datetime']<end_date)]['y_240d'])

In [135]:
start_rank=11
end_rank=15
start_date='1900-01-01'
end_date='2021-04-29'
df_train[(df_train['rank_of_average_rank']>=start_rank)
                &(df_train['rank_of_average_rank']<=end_rank)
                 &(df_train['datetime']>start_date)
                 &(df_train['datetime']<end_date)]

,instrument,datetime,$DWJZ,$LJJZ,y_1d,y_5d,y_20d,y_60d,y_120d,y_240d,...,his_480d,his_720d,rank_20d,rank_60d,rank_120d,rank_240d,rank_480d,rank_720d,average_rank,rank_of_average_rank
726,000001,2008-01-03,2.2460,3.4270,0.437697,2.713742,-4.260290,-11.438575,-16.078207,-23.519112,...,203.006180,206.255600,79.0,76.0,129.0,129.0,67.0,54.0,1945.0,15.0
727,000001,2008-01-04,2.2610,3.4420,0.900643,2.643814,-5.026144,-10.371874,-15.891922,-23.939568,...,204.601761,207.047256,71.0,83.0,131.0,128.0,69.0,52.0,1939.0,14.0
731,000001,2008-01-10,2.3390,3.5200,0.369319,-2.414774,-5.227273,-12.301140,-15.710226,-24.715904,...,215.695068,221.167908,90.0,49.0,131.0,120.0,68.0,51.0,1856.0,14.0
732,000001,2008-01-11,2.3520,3.5330,0.198131,-1.981317,-6.057175,-14.690066,-16.161905,-24.568356,...,213.487137,223.831360,103.0,37.0,133.0,123.0,67.0,50.0,1888.0,13.0
737,000001,2008-01-18,2.2820,3.4630,-2.252384,-1.414959,-4.187120,-15.737802,-16.142073,-23.678896,...,201.655045,210.304657,83.0,50.0,114.0,108.0,67.0,52.0,1714.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10709995,762001,2016-03-07,1.7191,2.3491,-0.830106,-2.664856,3.345959,2.745726,7.964749,10.165587,...,111.250015,128.934799,456.0,90.0,927.0,7.0,9.0,12.0,6945.0,13.0
10710014,762001,2016-04-01,1.7551,2.3851,1.786095,1.064952,1.605804,1.857368,4.272362,8.670506,...,116.787842,133.489960,1385.0,45.0,562.0,5.0,9.0,21.0,10837.0,12.0
10710029,762001,2016-04-25,1.7486,2.3786,0.126125,1.883463,0.084087,5.440176,6.524847,8.336843,...,127.269249,123.133202,380.0,268.0,102.0,9.0,9.0,18.0,4091.0,11.0
10710030,762001,2016-04-26,1.7516,2.3816,0.167962,1.864302,-0.650816,4.648141,6.147137,7.944247,...,124.933891,125.594398,434.0,281.0,107.0,7.0,10.0,21.0,4499.0,15.0


In [106]:
simulation_240d(1,10,'2012-01-01')

7.757286

In [107]:
simulation_240d(2,11,'2012-01-01')

7.8426085

In [108]:
simulation_240d(1,5,'2012-01-01')

7.9922957

In [109]:
simulation_240d(11,20,'2012-01-01')

7.682761

In [110]:
simulation_240d(11,15,'2012-01-01')

7.777929

In [111]:
simulation_240d(21,30,'2012-01-01')

7.891916

# 2013以后

In [112]:
simulation_240d(1,10,'2013-01-01')

7.773321

In [113]:
simulation_240d(1,10,'2014-01-01')

7.8573985

In [114]:
simulation_240d(1,10,'2015-01-01')

-0.48472142

In [115]:
simulation_240d(1,10,'2016-01-01')

2.57947

In [116]:
simulation_240d(1,10,'2017-01-01')

-0.26562902

# 定投

# Step 4 预测未来最优潜力组合

In [136]:
# 根据单月盈利排名选择
def get_list_by_rank(df_test, start_rank,end_rank,date='2021-04-29'): 
    df_left = df_test[(df_test['rank_of_average_rank']>=start_rank)&(df_test['rank_of_average_rank']<=end_rank)&(df_test['datetime']==date)][['instrument','datetime','average_rank','rank_of_average_rank']].sort_values(by=['rank_of_average_rank'])
    df_fund_code = read_fund_code_from_DB()
    return pd.merge(df_left, df_fund_code, how='left', left_on='instrument', right_on='code')

get_list_by_rank(df_new,1,5, date='2021-04-29')

,instrument,datetime,average_rank,rank_of_average_rank,index,code,ename,cname,type
0,005296,2021-04-29,2080.0,1.0,3876,005296,NHFCHHA,南华丰淳混合A,混合型-偏股
1,005297,2021-04-29,2202.0,2.0,3877,005297,NHFCHHC,南华丰淳混合C,混合型-偏股
2,090020,2021-04-29,2315.0,3.0,10609,090020,DCJKCYHH,大成健康产业混合,混合型-偏股
3,004040,2021-04-29,2612.0,4.0,2964,004040,JYYLJKCYA,金鹰医疗健康产业A,股票型
4,004041,2021-04-29,2772.0,5.0,2965,004041,JYYLJKCYC,金鹰医疗健康产业C,股票型


In [137]:
# 根据编码和日期选择
def get_list_by_no(df_test,no,date='2021-04-29'): 
    df_left = df_test[(df_test['instrument'] == no)&(df_test['datetime']==date)][['instrument','datetime','average_rank','rank_of_average_rank']].sort_values(by=['rank_of_average_rank'])
    df_fund_code = read_fund_code_from_DB()
    return pd.merge(df_left, df_fund_code, how='left', left_on='instrument', right_on='code')
get_list_by_no(df_test,no='001532',date='2021-04-23')

,instrument,datetime,average_rank,rank_of_average_rank,index,code,ename,cname,type
0,001532,2021-04-23,7339.0,21.0,1162,001532,HAWTJKHH,华安文体健康混合,混合型-灵活


In [138]:
df_test.shape

(6297475, 20)

## 机会识别
连续几个月的排名都在100名之内的基金: 
过去4个月都在前100，建议购买

In [139]:
topN = 100
date_list = ['2019-11-22','2020-11-24','2021-05-24','2021-08-05']
df_date_list = pd.DataFrame({'date':date_list})

def get_topN_for_months(df_test, topN,df_date_list):
    # condider Date
    df_date_list = pd.DataFrame({'date':date_list})
    df_date_list['date'] = pd.to_datetime(df_date_list['date'])
    df_left = df_test[['instrument','datetime','average_rank','rank_of_average_rank']].sort_values(by=['rank_of_average_rank'])
    df_left = pd.merge(df_left, df_date_list, how='inner', left_on='datetime', right_on='date')


    # consider TopN
    df_max_rank = pd.DataFrame()
    df_max_rank["max_rank"] = df_left.groupby(['instrument'])["rank_of_average_rank"].max()
    df_max_rank = df_max_rank.reset_index()
    df_max_rank = df_max_rank[df_max_rank["max_rank"]<=topN]
    df_left = pd.merge(df_left, df_max_rank, how='inner', on='instrument')
    df_fund_code = read_fund_code_from_DB()
    return pd.merge(df_left, df_fund_code, how='left', left_on='instrument', right_on='code')

df_result = get_topN_for_months(df_test, topN,df_date_list)
df_result
df_result[df_result['date'] == '2021-08-05' ]

,instrument,datetime,average_rank,rank_of_average_rank,date,max_rank,index,code,ename,cname,type
2,001704,2021-08-05,548.0,2.0,2021-08-05,99.0,1291,001704,GTRYJBLHPZHH,国投瑞银进宝灵活配置混合,混合型-灵活
6,004745,2021-08-05,753.0,7.0,2021-08-05,75.0,3430,004745,CSCXQDLHPZHH,长盛创新驱动灵活配置混合,混合型-灵活
10,004925,2021-08-05,3594.0,44.0,2021-08-05,45.0,3575,004925,CXDTHBHYLHGP,长信低碳环保行业量化股票,股票型
14,005668,2021-08-05,4487.0,60.0,2021-08-05,60.0,4196,005668,RTXNYQCZTJXHHA,融通新能源汽车主题精选混合A,混合型-灵活
18,005669,2021-08-05,572.0,3.0,2021-08-05,52.0,4197,005669,QHKYGYSYGP,前海开源公用事业股票,股票型
22,005821,2021-08-05,5021.0,72.0,2021-08-05,72.0,4326,005821,WJXJYLTQYHH,万家新机遇龙头企业混合,混合型-灵活
26,005825,2021-08-05,5663.0,88.0,2021-08-05,88.0,4330,005825,SWLXZNQDGP,申万菱信智能驱动股票,股票型
30,005977,2021-08-05,3686.0,46.0,2021-08-05,46.0,4465,005977,ZXBCZXHHA,中信保诚至兴混合A,混合型-灵活
34,005978,2021-08-05,3920.0,49.0,2021-08-05,49.0,4466,005978,ZXBCZXHHC,中信保诚至兴混合C,混合型-灵活
38,501057,2021-08-05,3822.0,47.0,2021-08-05,47.0,11980,501057,HTFZZXNYQCA,汇添富中证新能源汽车A,指数型-股票


In [142]:
get_list_by_no(df_new,'001532',date='2021-05-24')

,instrument,datetime,average_rank,rank_of_average_rank,index,code,ename,cname,type
0,001532,2021-05-24,3484.0,23.0,1162,001532,HAWTJKHH,华安文体健康混合,混合型-灵活


In [140]:
get_list_by_no(df_test,'001532',date='2019-11-21')

,instrument,datetime,average_rank,rank_of_average_rank,index,code,ename,cname,type
0,001532,2019-11-21,NaN,NaN,1162,001532,HAWTJKHH,华安文体健康混合,混合型-灵活


### 持仓中的表现情况，淘汰比较差的基金

In [93]:
my_fund_list = [
'002943',
'180012',
'001532',
'162605',
'163406',
'005233',
'000083',
'005827',
'008099',
'004997',
'164205',
'001557',
'502056',
'501057',
'000628',
'163417',
'001410',
'001298',
'003567',
'005167',
'003096','001632','004925','161125']
df_no_list = pd.DataFrame({'no':my_fund_list})

date_list = ['2021-04-23','2021-05-24','2021-06-23','2021-08-06']

def get_list_by_no_list(df_test, df_no_list,date_list):   
    # condider Date
    df_date_list = pd.DataFrame({'date':date_list})
    df_date_list['date'] = pd.to_datetime(df_date_list['date'])
    df_left = df_test[['instrument','datetime','average_rank','rank_of_average_rank']].sort_values(by=['rank_of_average_rank'])
    df_left = pd.merge(df_left, df_date_list, how='inner', left_on='datetime', right_on='date')
    
    df_left = df_left[['instrument','datetime','average_rank','rank_of_average_rank']].sort_values(by=['rank_of_average_rank'])
    df_left = pd.merge(df_left, df_no_list, how='inner', left_on='instrument', right_on='no')
    
        # consider TopN
    df_max_rank = pd.DataFrame()
    df_max_rank["mean_rank"] = df_left.groupby(['instrument'])["rank_of_average_rank"].mean()
    df_max_rank = df_max_rank.reset_index().sort_values(by=['mean_rank'])
    
    df_fund_code = read_fund_code_from_DB()
    return pd.merge(df_max_rank, df_fund_code, how='left', left_on='instrument', right_on='code')

df_result = get_list_by_no_list(df_test,df_no_list,date_list)
df_result

,instrument,mean_rank,index,code,ename,cname,type
0,001298,9.000000,980,001298,JYMZXXHH,金鹰民族新兴混合,混合型-灵活
1,501057,26.000000,11980,501057,HTFZZXNYQCA,汇添富中证新能源汽车A,指数型-股票
2,003567,34.250000,2679,003567,HXHYJQHH,华夏行业景气混合,混合型-偏股
3,001532,54.500000,1162,001532,HAWTJKHH,华安文体健康混合,混合型-灵活
4,004925,165.250000,3575,004925,CXDTHBHYLHGP,长信低碳环保行业量化股票,股票型
5,002943,172.000000,2236,002943,GFDYZHH,广发多因子混合,混合型-灵活
6,001557,765.250000,1183,001557,THZZ500ZSZQC,天弘中证500指数增强C,指数型-股票
7,001410,930.250000,1071,001410,XDAYXNYCYGP,信达澳银新能源产业股票,股票型
8,003096,933.500000,2335,003096,ZOYLJKHHC,中欧医疗健康混合C,混合型-偏股
9,161125,1014.666667,11082,161125,YFDBP500ZSRMB,易方达标普500指数人民币,QDII
